# Requirements

In [1]:

from langchain_groq import ChatGroq
import streamlit as st
from langchain_core.prompts import ChatPromptTemplate 
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
import pdfplumber


# LLM Model

In [2]:

llama = ChatGroq(
    model="LLaMA3-70B-8192",
    groq_api_key='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',
    temperature=0.0
)

# Navigation

In [ ]:
# Title
st.title("AI EXAM PREPPER by PARTH &mdash;:books::magic_wand:")

# Pages defined Home, Question, Answer, Concept-Learning Plan
pages = {
    "Home": [
        st.Page("Home.py", title="Welcome!"),
    ],
    "Question": [
        st.Page("Que.py", title="Upload your PYQ papers."),
    ],
    "Answer": [
        st.Page("Answer.py", title="Enter the question text file."),
    ],
    "Concept-Learning Plan": [
        st.Page("PL.py", title="Enter the question text file."),
    ],
}

#Clear history button
st.sidebar.markdown("""
                    **This button will clear the response history.** &mdash; :point_down:\n 
                    Recommended if you are switching pages.
                    """)
if st.sidebar.button("Clear History"):
    st.session_state.clear()

# Navigation page execution
pg = st.navigation(pages)
pg.run()

# Home page

In [9]:

# Introduction Text
st.markdown("""
### Hello students! Hope you find this app when your exam is near.\n
This app will make your exam preparations smooth as butter!\n
        Functionalities; this app provides:\n
            1. Question
            2. Answer
            3. Concept-Learning Plan
""")

# How to use section
st.markdown("**How to use?**")

with st.expander("Instructions to follow..."):
    st.markdown("""
Open the **Question** page -> Upload pyq papers (only in pdf format) -> Download the ***Question_response*** text file.\n 
You can you this file for both Answer page and Concept-Learning Plan for getting responses.\n
                """)
    

# Description
st.markdown("**Description about app's functions**")

with st.expander("About Question..."):
    st.markdown("""
    **Question** is a place where you upload your Past Year Question papers and get your imp questions in seconds.\n
    Input: Question papers(one or more) **Format:** PDF\n
    Task: It will generate you a list of questions which are repeated and catagorize it on concept base.\n
    Output: List of imp questions. Also, you can download it as a file.
    """)

with st.expander("About Answer..."):
    st.markdown("""
In **Answer** page, you will get brief and informative answer for any questions provided. \n
Input: The ***Question_response*** text file from the **Question** page.\n
Task: Generation of answers with conceptual understanding.\n
Output: Answer text file to download.                          
""")
    
with st.expander("About Concept-Learning Plan..."):
    st.markdown("""
In **Concept-Learning Plan** page, you will get a guided study plan to kickstart your studying.\n
Input: The ***Question_response*** text file from the **Question** page.\n
Task: Generation of a Study plan according to your imp concepts covered in the exam with a flow chart to visualize it better.\n
Output: Plan text file to download.                                               
""")

# Quesion Page

In [ ]:

# Setting up chains

qchain= ( ChatPromptTemplate.from_template("Create and Provide a list of the repeated questions 'or' similar conceptual questions with their concept from the {base_response}. Also, If any questions are repeated then state their repetitions.")
                      | llama
                      | StrOutputParser()
                      #| {"q_response": RunnablePassthrough()}
                      
            )
basechain = ( ChatPromptTemplate.from_template("you are a expert analyst. Your task is to Analyzize these question papers {res} and find all the questions from each question paper according to their concept.")
                      | llama
                      | StrOutputParser()
                      |{"base_response": RunnablePassthrough()}
                      
            )

mainchainq = (
     basechain
     | qchain
)

# Load Groq compiled LLaMA model (replace with your actual model path)
@st.cache_resource


# Function to download response
def download_response_as_pdf(bot_response):
    st.download_button(
        label="Download as file",
        data=bot_response,
        file_name="Question_response.txt"
    )
     
if "messages" not in st.session_state:
    st.session_state.messages = []

# Getting user input
uploaded_files = st.file_uploader(
        "Upload your PYQ papers below. (Only .pdf is allowed)", accept_multiple_files=True
    )

# Function to extract text from files
def extract():
    
        extracted_text = []
        for file in uploaded_files:
            with pdfplumber.open(file) as pdf:
                for page in pdf.pages:
                    extracted_text.append(page.extract_text())
                   
        return extracted_text
res= extract()

# Submit button action
if st.button("Submit"):
     message = st.chat_message("assistant")
     bot_response = mainchainq.invoke(res)
     st.session_state.messages.append({"role": "assistant", "content": bot_response})
     download_response_as_pdf(bot_response)

# # Display chat history
for message in st.session_state.messages:
    if message["role"] == "assistant":
        st.write(f"bot: {message['content']}")

# Answer Page

In [61]:

# Setting up chains
achain= ( ChatPromptTemplate.from_template("You are an intelligent ai made for assisting students for completing their assingnments. Generate informative answers  of minimum 700 words for every question in this assignment {que}")
                      | llama
                      | StrOutputParser()
            )
basechain = ( ChatPromptTemplate.from_template("you are a expert analyst. Your task is to Analyzize these questions {que} with their concepts.")
                      | llama
                      | StrOutputParser()
                      |{"base_response": RunnablePassthrough()}
)
anschain= (
    basechain
    | achain
)

# Getting user input
uploaded_files = st.file_uploader(
        "Note: Only upload the Question_response.txt file", accept_multiple_files=True
    )

# Function to extract text from files
def questions():
    decoded_text = []
    for file in uploaded_files:
        file_bytes = file.read()
        decoded_text = file_bytes.decode("utf-8")
    return decoded_text
que = questions()

# Function to download response
def download_response_as_pdf(bot_response):
    st.download_button(
        label="Download as file",
        data=bot_response,
        file_name="Ans_response.txt"
    )

if "messages" not in st.session_state:
    st.session_state.messages = []

# Submit button action
if st.button("Submit"):
     message = st.chat_message("assistant")
     bot_response = achain.invoke(que)
     st.session_state.messages.append({"role": "assistant", "content": bot_response})
     download_response_as_pdf(bot_response)

for message in st.session_state.messages:
    if message["role"] == "assistant":
        st.write(f"bot: {message['content']}")

# Concept-Learning Plan

In [121]:

# Setting up chains
cchain= ( ChatPromptTemplate.from_template("you are a expert analyst. your task is to Create a hierarchal plan for studying these concepts {concepts} and represent it with flow chart. Also, explain the your created plan. Guide and encourage students to follow your plan.")
                      | llama
                      | StrOutputParser()
            )
basechain = ( ChatPromptTemplate.from_template("you are a expert analyst. Your task is to Analyzize the given text {txt} and find out the concepts covered in it.")
                      | llama
                      | StrOutputParser()
                      |{"concepts": RunnablePassthrough()}
)
conchain= (
    basechain
    | cchain
)

# Getting user input
uploaded_files = st.file_uploader(
        "Note: Only upload the Question_response.txt file", accept_multiple_files=True
    )

# Function to extract text from files
def concept():
    decoded_text = []
    for file in uploaded_files:
        file_bytes = file.read()
        decoded_text = file_bytes.decode("utf-8")
    return decoded_text
txt = concept()

# Function to download response
def download_response_as_pdf(bot_response):
    st.download_button(
        label="Download as file",
        data=bot_response,
        file_name="Plan_response.txt",
        #mime="application/pdf"
    )

if "messages" not in st.session_state:
    st.session_state.messages = []

if st.button("Submit"):
     message = st.chat_message("assistant")
     bot_response = conchain.invoke(txt)
     st.session_state.messages.append({"role": "assistant", "content": bot_response})
     download_response_as_pdf(bot_response)

for message in st.session_state.messages:
    if message["role"] == "assistant":
        st.write(f"bot: {message['content']}")